In [1]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)

None


In [2]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://3.238.27.77:7687",
    username="neo4j",
    password="army-acids-fashion"
)

result = graph.query("""
MATCH (m:Movie{title: 'Toy Story'}) 
RETURN m.title, m.plot, m.poster
""")

print(result)

[{'m.title': 'Toy Story', 'm.plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.", 'm.poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg'}]


In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

OPENAI_API_KEY = OPENAI_API_KEY

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

embedding_provider = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

graph = Neo4jGraph(
    url="bolt://3.238.27.77:7687",
    username="neo4j",
    password="army-acids-fashion"
)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name="moviePlots",
    embedding_node_property="plotEmbedding",
    text_node_property="plot",
)

plot_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=movie_plot_vector.as_retriever(),
    verbose=True,
    return_source_documents=True
)

response = plot_retriever.invoke(
    {"query": "A movie where a mission to the moon goes wrong"}
)

print(response)



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'A movie where a mission to the moon goes wrong', 'result': 'The movie you are referring to is likely "2001: A Space Odyssey" where a mission to the Moon goes awry after the discovery of a mysterious artificial object buried beneath the Lunar surface.', 'source_documents': [Document(metadata={'budget': 12000000, 'movieId': '924', 'tmdbId': '62', 'imdbVotes': 407650, 'runtime': 149, 'countries': ['USA', ' UK'], 'imdbId': '0062622', 'url': 'https://themoviedb.org/movie/62', 'released': '1968-05-15', 'languages': ['English', ' Russian'], 'imdbRating': 8.3, 'title': '2001: A Space Odyssey', 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/zmmYdPa8Lxx999Af9vnVP4XQ1V6.jpg', 'year': 1968, 'revenue': 68700000}, page_content='Humanity finds a mysterious, obviously artificial object buried beneath the Lunar surface and, with the intelligent computer H.A.L. 9000, sets off on a quest.'), Document(metadata={'budget': 5200000